#**1 Import**

In [ ]:
#Import generali
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import os
import shutil
import cv2
import numpy as np
import random
from random import randint
from PIL import Image
import tensorflow as tf

In [ ]:
#Import usati per la classificazione
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression    #L1 regression, L2 regression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
#Import usati per la classificazione con reti neurali 
from tensorflow import keras
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPool2D, Dropout, BatchNormalization,Activation,InputLayer,GlobalAveragePooling2D, ZeroPadding2D,Lambda
from keras import Model

In [ ]:
#Monto il contenuto di google drive su colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**2 Classificazione con modelli di ML (senza data augmentation)**

Adesso diamo in input le features estratte sia da ResNet50 che da VGG16 a diversi modelli di classificazione e vediamo con quali di questi otteniamo l'accuracy più alta.

In [ ]:
FeatureExtractionVGG16 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/file/features_extraction/FeatureExtractionVGG16.csv')
FeatureExtractionResNet50 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/file/features_extraction/FeatureExtractionResNet50.csv')

In [ ]:
#Assegniamo alla X il dataframe con le features estratte eliminando la colonna contenente le labels(diag)

#Senza l'argomento inplace = True dobbiamo assegnare il risultato ottenuto perchè la viariabile originale non viene modificata
XResNet50 = FeatureExtractionResNet50.drop(['diag'], axis=1)

XVGG16 = FeatureExtractionVGG16.drop(['diag'], axis=1)

#la Y è uguale per entrambi i dataframe e in questo caso la prendiamo da ResNet50
Y = FeatureExtractionResNet50['diag']

##2.1 Classificazione con features estratte da ResNet50

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
XResNet50.drop(["Unnamed: 0"], axis=1, inplace=True)

###2.1.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]
  
  
   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)
   
     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac

   
     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.98
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.84 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.82 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.84
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.79 



In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.89
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.80 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.80 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.53
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.51 



###2.1.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train) 
   
   train_ac += decision_tree.score(X_train,y_train)
     
   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train)) 
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.72



###2.1.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]
  
  
  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)
   
   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac

   
   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.83
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.84
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.82


###2.1.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XResNet50, Y, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']
  
  
  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)
   
   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac

   
   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.82
Training accuracy of SVM classifier with kernel = poly : 0.78
Test accuracy of SVM classifier with kernel = poly : 0.75
Training accuracy of SVM classifier with kernel = rbf : 0.77
Test accuracy of SVM classifier with kernel = rbf : 0.74
Training accuracy of SVM classifier with kernel = sigmoid : 0.55
Test accuracy of SVM classifier with kernel = sigmoid : 0.50


##2.2 Classificazione con features estratte da VGG16

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
XVGG16.drop(["Unnamed: 0"], axis=1, inplace=True)

###2.2.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]
  
  
   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)
   
     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac

   
     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

###2.2.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train) 
   
   train_ac += decision_tree.score(X_train,y_train)
     
   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train)) 
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.84



###2.2.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]
  
  
  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)
   
   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac

   
   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.69
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.71
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.70


###2.2.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(XVGG16, Y, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']
  
  
  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)
   
   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac

   
   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.80
Training accuracy of SVM classifier with kernel = poly : 0.72
Test accuracy of SVM classifier with kernel = poly : 0.70
Training accuracy of SVM classifier with kernel = rbf : 0.69
Test accuracy of SVM classifier with kernel = rbf : 0.61
Training accuracy of SVM classifier with kernel = sigmoid : 0.67
Test accuracy of SVM classifier with kernel = sigmoid : 0.68


#**3 Classificazione con modelli di ML (con data augmentation)**

In [ ]:
FeatureExtractionDataAugmentationVGG16 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/file/features_extraction/FeatureExtractionDataAugmentationVGG16.csv')
FeatureExtractionDataAugmentationResNet50 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/file/features_extraction/FeatureExtractionDataAugmentationResNet50.csv')

In [ ]:
#Assegniamo alla X il dataframe con le features estratte eliminando la colonna contenente le labels(diag)

#Senza l'argomento inplace = True dobbiamo assegnare il risultato ottenuto perchè la viariabile originale non viene modificata
X_ResNet50 = FeatureExtractionDataAugmentationResNet50.drop(['diag'], axis=1)

X_VGG16 = FeatureExtractionDataAugmentationVGG16.drop(['diag'], axis=1)

#la Y è uguale per entrambi i dataframe e in questo caso la prendiamo da ResNet50
Y_da = FeatureExtractionDataAugmentationResNet50['diag']

##3.1 Classificazione con features estratte da ResNet50

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
X_ResNet50.drop(["Unnamed: 0"], axis=1, inplace=True)

###3.1.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]
  
  
   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)
   
     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac

   
     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.97
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.82 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.99
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.81 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.83
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.80 



In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.93
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.78 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.76 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.65
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.63 



###3.1.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train) 
   
   train_ac += decision_tree.score(X_train,y_train)
     
   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train)) 
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.67



###3.1.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]
  
  
  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)
   
   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac

   
   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.76
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.76
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.74


###3.1.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_ResNet50, Y_da, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']
  
  
  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)
   
   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac

   
   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.77
Training accuracy of SVM classifier with kernel = poly : 0.80
Test accuracy of SVM classifier with kernel = poly : 0.77
Training accuracy of SVM classifier with kernel = rbf : 0.79
Test accuracy of SVM classifier with kernel = rbf : 0.75
Training accuracy of SVM classifier with kernel = sigmoid : 0.74
Test accuracy of SVM classifier with kernel = sigmoid : 0.72


##3.2 Classificazione con features estratte da VGG16

In [ ]:
#Rimuoviamo nuovamente la colonna unnamed: 0, che compare ogni volta che andiamo a rileggere il file csv
X_VGG16.drop(["Unnamed: 0"], axis=1, inplace=True)

###3.2.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 100
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]
  
  
   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)
   
     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac

   
     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.99
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.81 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.78 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.86
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.78 



/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.98
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.77 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.78 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.52
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.54 



###3.2.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 

In [ ]:
RS_NUM = 100
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train) 
   
   train_ac += decision_tree.score(X_train,y_train)
     
   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train)) 
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.67



###3.2.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]
  
  
  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)
   
   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac

   
   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.72
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.72
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.71


###3.2.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 100
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X_VGG16, Y_da, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']
  
  
  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)
   
   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac

   
   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.79
Training accuracy of SVM classifier with kernel = poly : 0.81
Test accuracy of SVM classifier with kernel = poly : 0.76
Training accuracy of SVM classifier with kernel = rbf : 0.79
Test accuracy of SVM classifier with kernel = rbf : 0.73
Training accuracy of SVM classifier with kernel = sigmoid : 0.68
Test accuracy of SVM classifier with kernel = sigmoid : 0.67


#**4 Concatenazione delle features ottenute senza data augmentation e con data augmentation**

Applichiamo questo procedimento solo a ResNet50

In [ ]:
#Otteniamo le features estratte da ResNet50 con data augmentation e senza data augmentation 
FeatureExtractionResNet50 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/file/features_extraction/FeatureExtractionResNet50.csv')
FeatureExtractionDataAugmentationResNet50 = pd.read_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/file/features_extraction/FeatureExtractionDataAugmentationResNet50.csv')


In [ ]:
concat_dataframe = FeatureExtractionResNet50.append(FeatureExtractionDataAugmentationResNet50)
concat_dataframe.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
concat_dataframe.to_csv('/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/file/features_extraction/ConcatFeatureResNet50.csv')

In [ ]:
X = concat_dataframe.drop(['diag'], axis=1)

Y = concat_dataframe['diag']

###4.1.1 Classificazione con Logistic Regression

Definiamo una funzione che ci permette di calcolare l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro C

In [ ]:
def logistic_regression_ac(model):
  Cvalues_to_accuracy_train = {1 : 0, 100 : 0, 0.01 : 0}
  Cvalues_to_accuracy_test = {1 : 0, 100 : 0, 0.01 : 0}
  RS_NUM = 10
  for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)
   C_values = [1, 100, 0.01]
  
  
   for value in C_values:

     logistic_regression = model
     logistic_regression.C=value
     logistic_regression.fit(X_train, y_train)
   
     train_ac = logistic_regression.score(X_train,y_train)
     Cvalues_to_accuracy_train[value]  += train_ac

   
     test_ac = logistic_regression.score(X_test,y_test)
     Cvalues_to_accuracy_test[value]  += test_ac


  for value in C_values:
     media_train = Cvalues_to_accuracy_train[value] / RS_NUM
     media_test = Cvalues_to_accuracy_test[value] / RS_NUM
     print("Training accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f}".format(value, media_train))
     print("Test accuracy of "+ 'logistic_regression with penalty = ' + model.penalty + " and with C={:.2f}: {:.2f} \n".format(value, media_test))


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l2_penalty
logistic_regression_ac(LogisticRegression())

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Training accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.97
Test accuracy of logistic_regression with penalty = l2 and with C=1.00: 0.90 

Training accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.99
Test accuracy of logistic_regression with penalty = l2 and with C=100.00: 0.90 

Training accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.84
Test accuracy of logistic_regression with penalty = l2 and with C=0.01: 0.84 



/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Calcoliamo l'accuracy media della logist_regression con l1_penalty
logistic_regression_ac(LogisticRegression(penalty = 'l1', solver='liblinear'))

Training accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.94
Test accuracy of logistic_regression with penalty = l1 and with C=1.00: 0.83 

Training accuracy of logistic_regression with penalty = l1 and with C=100.00: 1.00
Test accuracy of logistic_regression with penalty = l1 and with C=100.00: 0.89 

Training accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.69
Test accuracy of logistic_regression with penalty = l1 and with C=0.01: 0.68 



###4.1.2 Classificazione con Decision Tree

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 

In [ ]:
RS_NUM = 10
train_ac = 0
test_ac = 0
for i in range(RS_NUM):
   rs = random.randint(1, 100)
   X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)

   decision_tree = DecisionTreeClassifier()
   decision_tree.fit(X_train, y_train) 
   
   train_ac += decision_tree.score(X_train,y_train)
     
   test_ac += decision_tree.score(X_test,y_test)


media_train = train_ac / RS_NUM
media_test = test_ac/ RS_NUM
print("Training accuracy with decision tree classifier: {:.2f}".format(media_train)) 
print("Test accuracy with decision tree classifier: {:.2f}\n".format(media_test))

Training accuracy with decision tree classifier: 1.00
Test accuracy with decision tree classifier: 0.79



###4.1.3 Classificazione con Random Forest

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i tre valori diversi dell'iperparametro n_estimators

In [ ]:
Nvalues_to_accuracy_train = {100 : 0, 80 : 0, 50 : 0}
Nvalues_to_accuracy_test = {100 : 0, 80 : 0, 50 : 0}
RS_NUM = 10
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)
  n_estimators = [100, 80, 50]
  
  
  for estimator in n_estimators:

   rnd_forest = RandomForestClassifier(n_estimators= estimator)
   rnd_forest.fit(X_train, y_train)
   
   train_ac = rnd_forest.score(X_train,y_train)
   Nvalues_to_accuracy_train[estimator]  += train_ac

   
   test_ac = rnd_forest.score(X_test,y_test)
   Nvalues_to_accuracy_test[estimator]  += test_ac

for estimator in n_estimators:
  media_train = Nvalues_to_accuracy_train[estimator] / RS_NUM
  media_test = Nvalues_to_accuracy_test[estimator] / RS_NUM
  print("Training accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_train))
  print("Test accuracy of random forest classifier with n_estimators = {:d} : {:.2f}".format(estimator, media_test))

Training accuracy of random forest classifier with n_estimators = 100 : 1.00
Test accuracy of random forest classifier with n_estimators = 100 : 0.84
Training accuracy of random forest classifier with n_estimators = 80 : 1.00
Test accuracy of random forest classifier with n_estimators = 80 : 0.80
Training accuracy of random forest classifier with n_estimators = 50 : 1.00
Test accuracy of random forest classifier with n_estimators = 50 : 0.82


###4.1.4 Classificazione con SVM

Calcoliamo l'accuracy media sia sul training set che sul test set considerando dieci valori diversi di random state (ottenuti in maniera randomica). 
L'accuracy media viene calcolata per i 4 valori diversi dell'iperparametro kernel

In [ ]:
kernel_to_accuracy_train = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
kernel_to_accuracy_test = {'linear' : 0, 'poly' : 0, 'rbf' : 0, 'sigmoid' : 0}
RS_NUM = 10
for i in range(RS_NUM):
  rs = random.randint(1, 100)
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 20, random_state = rs)
  kernels = ['linear', 'poly', 'rbf', 'sigmoid']
  
  
  for value in kernels:

   clf = SVC(kernel = value)
   clf.fit(X_train, y_train)
   
   train_ac = clf.score(X_train,y_train)
   kernel_to_accuracy_train[value]  += train_ac

   
   test_ac = clf.score(X_test,y_test)
   kernel_to_accuracy_test[value]  += test_ac

for value in kernels:
  media_train = kernel_to_accuracy_train[value] / RS_NUM
  media_test = kernel_to_accuracy_test[value] / RS_NUM
  print("Training accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_train))
  print("Test accuracy of SVM classifier with kernel = {:s} : {:.2f}".format(value, media_test))

Training accuracy of SVM classifier with kernel = linear : 1.00
Test accuracy of SVM classifier with kernel = linear : 0.81
Training accuracy of SVM classifier with kernel = poly : 0.81
Test accuracy of SVM classifier with kernel = poly : 0.78
Training accuracy of SVM classifier with kernel = rbf : 0.80
Test accuracy of SVM classifier with kernel = rbf : 0.76
Training accuracy of SVM classifier with kernel = sigmoid : 0.75
Test accuracy of SVM classifier with kernel = sigmoid : 0.73


#**5 Classificazione con un modello preaddestrato di ResNet50**

In [ ]:
directory = '/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/file/DatasetLabel1'
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='training',
)

Found 720 files belonging to 2 classes.
Using 576 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='validation',
)

Found 720 files belonging to 2 classes.
Using 144 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['Dysgraphic', 'No Dysgraphic']


In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(432,288,3),
                   pooling='avg',classes=2,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(1, activation='sigmoid'))

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 module_wrapper (ModuleWrapp  (None, 2048)             0         
 er)                                                             
                                                                 
 module_wrapper_1 (ModuleWra  (None, 512)              1049088   
 pper)                                                           
                                                                 
 module_wrapper_2 (ModuleWra  (None, 1)                513       
 pper)                                                           
                                                                 
Total params: 24,637,313
Trainable params: 1,049,601
Non-trainable params: 23,587,712
____________________________________

In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate = 0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=15
history = resnet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15
18/18 [==============================] - 382s 19s/step - loss: 1.0042 - accuracy: 0.5365 - val_loss: 0.6609 - val_accuracy: 0.6250
Epoch 2/15
18/18 [==============================] - 343s 19s/step - loss: 0.5897 - accuracy: 0.6962 - val_loss: 0.6144 - val_accuracy: 0.6319
Epoch 3/15
18/18 [==============================] - 346s 19s/step - loss: 0.5285 - accuracy: 0.7222 - val_loss: 0.5722 - val_accuracy: 0.6597
Epoch 4/15
18/18 [==============================] - 330s 18s/step - loss: 0.4987 - accuracy: 0.7361 - val_loss: 0.5532 - val_accuracy: 0.6944
Epoch 5/15
18/18 [==============================] - 325s 18s/step - loss: 0.5121 - accuracy: 0.7309 - val_loss: 0.5604 - val_accuracy: 0.6806
Epoch 6/15
18/18 [==============================] - 346s 19s/step - loss: 0.4987 - accuracy: 0.7778 - val_loss: 0.6177 - val_accuracy: 0.7361
Epoch 7/15
18/18 [==============================] - 341s 19s/step - loss: 0.4988 - accuracy: 0.7708 - val_loss: 0.5870 - val_accuracy: 0.6736
Epoch 

#**6 Classificazione con rete neurale pre-addestra sul Dataset IAM**

Il dataset IAM contiene forme di testo scritte a mano in inglese.

Il modello e i relativi pesi pre-addestrati sono stati presi da qui:
https://github.com/diegocasmo/iam_writer_recognition


In [ ]:

model = Sequential()

# Define network input shape
model.add(ZeroPadding2D((1, 1), input_shape=(432, 288, 1)))

# CNN model - Building the model suggested in paper
model.add(Conv2D(filters= 32, kernel_size =(5,5), strides= (2, 2), padding='same', name='conv1'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))

model.add(Conv2D(filters= 64, kernel_size =(3, 3), strides= (1, 1), padding='same', name='conv2'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))

model.add(Conv2D(filters= 128, kernel_size =(3, 3), strides= (1, 1), padding='same', name='conv3'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool3'))


model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(512, name='dense1'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256, name='dense2'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [ ]:
model.load_weights('/content/drive/MyDrive/ProgettoML/model_weights.hdf5',by_name=True,skip_mismatch=True)

In [ ]:
model.trainable=False

In [ ]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_11 (ZeroPadd  (None, 434, 290, 1)      0         
 ing2D)                                                          
                                                                 
 conv1 (Conv2D)              (None, 217, 145, 32)      832       
                                                                 
 activation_50 (Activation)  (None, 217, 145, 32)      0         
                                                                 
 pool1 (MaxPooling2D)        (None, 108, 72, 32)       0         
                                                                 
 conv2 (Conv2D)              (None, 108, 72, 64)       18496     
                                                                 
 activation_51 (Activation)  (None, 108, 72, 64)       0         
                                                     

In [ ]:
new_model = Sequential()

new_model.add(model)


new_model.add(Dense(1, name='output'))
new_model.add(Activation('sigmoid'))

In [ ]:
new_model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_16 (Sequential)  (None, 256)               32075520  
                                                                 
 output (Dense)              (None, 1)                 257       
                                                                 
 activation_55 (Activation)  (None, 1)                 0         
                                                                 
Total params: 32,075,777
Trainable params: 257
Non-trainable params: 32,075,520
_________________________________________________________________


In [ ]:
directory = '/content/drive/MyDrive/ProgettoML/ProgettoDisgrafia/file/DatasetLabel1'
train = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='training',
)

Found 720 files belonging to 2 classes.
Using 576 files for training.


In [ ]:
valid = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(432, 288),
    seed=123,
    validation_split= 0.2,
    subset='validation',
)

Found 720 files belonging to 2 classes.
Using 144 files for validation.


In [ ]:
new_model.compile(optimizer=Adam(learning_rate = 0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=70
history = new_model.fit(
  train,
  validation_data=valid,
  epochs=epochs
)

Epoch 1/70
18/18 [==============================] - 15s 748ms/step - loss: 1.0871 - accuracy: 0.5139 - val_loss: 0.9526 - val_accuracy: 0.4722
Epoch 2/70
18/18 [==============================] - 15s 829ms/step - loss: 0.9369 - accuracy: 0.5052 - val_loss: 0.7084 - val_accuracy: 0.5625
Epoch 3/70
18/18 [==============================] - 16s 816ms/step - loss: 0.8227 - accuracy: 0.5573 - val_loss: 0.7191 - val_accuracy: 0.4583
Epoch 4/70
18/18 [==============================] - 14s 740ms/step - loss: 0.9492 - accuracy: 0.4965 - val_loss: 0.7208 - val_accuracy: 0.5278
Epoch 5/70
18/18 [==============================] - 14s 736ms/step - loss: 0.9306 - accuracy: 0.5052 - val_loss: 0.6909 - val_accuracy: 0.5625
Epoch 6/70
18/18 [==============================] - 14s 739ms/step - loss: 0.9322 - accuracy: 0.5052 - val_loss: 0.7149 - val_accuracy: 0.5208
Epoch 7/70
18/18 [==============================] - 16s 836ms/step - loss: 0.8334 - accuracy: 0.5451 - val_loss: 0.7247 - val_accuracy: 0.4097

In [ ]:
# scongeliamo i pesi dei layer del modello
model.trainable = True


new_model.compile(optimizer=keras.optimizers.Adam(1e-5),  
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])


history = new_model.fit(
  train,
  validation_data=valid,
  epochs=40
)

Epoch 1/40
18/18 [==============================] - 43s 2s/step - loss: 0.8172 - binary_accuracy: 0.5156 - val_loss: 0.6852 - val_binary_accuracy: 0.5069
Epoch 2/40
18/18 [==============================] - 40s 2s/step - loss: 0.7435 - binary_accuracy: 0.5503 - val_loss: 0.6889 - val_binary_accuracy: 0.5347
Epoch 3/40
18/18 [==============================] - 42s 2s/step - loss: 0.7590 - binary_accuracy: 0.5295 - val_loss: 0.6867 - val_binary_accuracy: 0.5278
Epoch 4/40
18/18 [==============================] - 42s 2s/step - loss: 0.7000 - binary_accuracy: 0.5208 - val_loss: 0.6879 - val_binary_accuracy: 0.5556
Epoch 5/40
18/18 [==============================] - 42s 2s/step - loss: 0.6802 - binary_accuracy: 0.5729 - val_loss: 0.6833 - val_binary_accuracy: 0.6111
Epoch 6/40
18/18 [==============================] - 40s 2s/step - loss: 0.7101 - binary_accuracy: 0.5451 - val_loss: 0.6783 - val_binary_accuracy: 0.5903
Epoch 7/40
18/18 [==============================] - 41s 2s/step - loss: 0.68